## 네이버 검색어 지정 및 기사(News) 크롤링

In [ ]:
import os
import sys
from urllib.parse import urlparse
import requests as req
search = "정보보안"
url = 'https://openapi.naver.com/v1/search/'+'news'+'?query='+search+'&display=3'
res = req.get(urlparse(url).geturl(),
headers={'X-Naver-Client-Id': 'ys1o3hhvs_IL8YTasnby',
'X-Naver-Client-Secret': 'wYUFA7vyAs'})
print(res.status_code)
print(res.text)

## 티스토리 블로그 자동 로그인 및 access token 발급 받기

In [39]:
import requests
import json
import re

# App 아이디 설정
client_id = 'a1f11d9eb54d53fcdb19da6026fa0fa7'

# Callback 설정
redirect_uri = 'https://kkamagistory.tistory.com'
oauth_url = "https://www.tistory.com/oauth/authorize?client_id="+client_id+"&redirect_uri="+redirect_uri+"&response_type=token"

#tistory account
user_id = 'karas639@gmail.com'
password = 'dwkimsec135@$^'

print(oauth_url)

res = requests.get(oauth_url)
print(res.status_code)
#print(res.headers['Set-Cookie'])
#kakao_cookie = res.headers['Set-Cookie'].replace("; path=/",'')
#print(kakao_cookie)
print(res.url)
headers = {
    'Accept' : 'text/html, application/xhtml+xml, image/jxr, */*',
    'Accept-Encoding' : 'gzip, deflate',
    'Accept-Language' : 'ko, en-US; q=0.8, en; q=0.6, zh-Hans-CN; q=0.4, zh-Hans; q=0.2',
    'Cache-Control' : 'no-cache',
    'Connection' : 'Keep-Alive',
    'Content-Type' : 'application/x-www-form-urlencoded',
    #'Cookie' : kakao_cookie,
    'Host' : 'www.tistory.com',
    'Referer' : res.url,
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko'
}
login_data = {
    'fp' : 'a1f11d9eb54d53fcdb19da6026fa0fa7',
    'keepLogin' : 'on',
    'loginId' : user_id,
    'password' : password,
    'redirectUrl' : res.url
}

res = requests.post('https://www.tistory.com/auth/login', headers=headers, data=login_data)
#print(res.url)
match = re.match('(.*?)access_token=(?P<access_token>.*?)&state=', res.url)
#print(match)
gd = match.groupdict()
access_token = gd['access_token']
print("access token : ", access_token)

def get_info():
    info_url = f'https://www.tistory.com/apis/blog/info?access_token={access_token}&output={output_type}'
    r =requests.get(info_url, headers=requests_headers)
    return r.text

def get_list(page_number):
    url = f'https://www.tistory.com/apis/post/list?access_token={access_token}&output={output_type}&blogName={blog_name}&page={page_number}'
    r = requests.get(url, headers = requests_headers)
    return r.text

def get_category():
    url = f'https://www.tistory.com/apis/category/list?access_token={access_token}&output={output_type}&blogName={blog_name}'
    r = requests.get(url, headers=requests_headers)
    r = r.json()['tistory']['item']['categories']
    for i in r:
        print(f'{i["name"]} ({i["id"]})')
    return r

def get_write():
    title = "테스트" #제목 (필수)
    content = "글 내용" #글내용(필수)
    visibility = "3" #발행상태 0비공개-기본, 1보고,3발행
    category_id = "0" #카테고리 아이디 기본값 0
    published ="" #기본값 : 현재시간
    slogan = "" #문자주소
    tag = "" #태그 ,로 구분
    acceptComment = "" #댓글 허용 (0, 1 - 기본값)
    password = "" #보호글 비밀번호
    url = f'https://www.tistory.com/apis/post/write?access_token={access_token}&output={output_type}&blogName={blog_name}&title={title}&content={content}&visibility={visibility}&category={category_id}&slogan={slogan}&tag={tag}&acceptComment={acceptComment}&password={password}'
    r = requests.post(url)
    print(r, r.text)
    
def file_upload():
    files = {'uploadedfile': open(filepath, 'rb')}
    params = {'access_token': access_token, 'blogName': kkamagistory, 'targetUrl': blogName, 'output': 'json'}
    rd = requests.post('https://www.tistory.com/apis/post/attach', params=params, files=files)
 
    try:
        item = json.loads(rd.text)
        print(json.dumps(item, indent=4))
        print("----------------------------------------------")
        print(item["tistory"]["replacer"])
        print(item["tistory"]["url"])
        print(item["tistory"]["status"])
    except:
        print("Failed")
 
 
    get_category()

https://www.tistory.com/oauth/authorize?client_id=a1f11d9eb54d53fcdb19da6026fa0fa7&redirect_uri=https://kkamagistory.tistory.com&response_type=token
200
https://www.tistory.com/auth/login/?popup=1&redirectUrl=https%3A%2F%2Fwww.tistory.com%2Foauth%2Fauthorize%3Fclient_id%3Da1f11d9eb54d53fcdb19da6026fa0fa7%26redirect_uri%3Dhttps%3A%2F%2Fkkamagistory.tistory.com%26response_type%3Dtoken
access token :  3d995c313268cc34700ff3e67ba20efd_c81935382d3874d1fa5e6c9dbd1372a8


# 2020.6.15일 변경사항

### 참고 : https://godpeople.or.kr/board/4927670
-400에러가 발생하면서 "access_token 이 유효하지 않습니다."라는 에러가 발생하는 경우에 대한 패치

-원인, request로 처리하는 과정에서 해더 정보를 새롭게 체크하는 것 같습니다.

-해당 해더 정보를 추가하면 됩니다.

- 아래 참고

requests_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
}

 

requests.get(url, headers=requests_headers)